# Access your data in Azure Notebooks

Data is the lifeblood of notebooks. Jupyter itself provides only a runtime environment for a notebook, and thus to do intresting work you need to bring data in from elsewhere.

This notebook provides examples of different ways to import data, all in a format that you can run and experience directly.

- [Use curl to retrieve a file from GitHub](#curl)
- [Use a REST API to retrieve online data](#restapi)
- [Query an Azure SQL database](#azuresql)
- [Access Azure Table Storage](#tablestorage)
- [Access Azure Blobs](#blobs)
  - [Share access to Azure Storage through Shared Access Signatures](#sharedaccess)
- [Other Azure databases (references)](#otherdbs)

# Use `curl` to retrieve a file from the Internet <a name="curl"></a> 

In Python notebooks, you can invoke the command line using `!`, which allows you to download files directly from the Internet using a tool like `curl`, `wget`, and so on. For example, the following `curl` command downloads a file containing oil price data from GitHub, and stores it in the project as the file *oil_price_temp.csv*:

In [1]:
!curl https://raw.githubusercontent.com/petroleum101/figures/db46e7f48b8aab67a0dfe31696f6071fb7a84f1e/oil_price/oil_price.csv -o ../oil_price_temp.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5190  100  5190    0     0  24049      0 --:--:-- --:--:-- --:--:-- 24366


Once the file is in the project, you can load it using any suitable code. For example, you can load it into a pandas dataframe, after which you can work with it however you like:

In [2]:
import pandas
dataframe_file = pandas.read_csv('../oil_price_temp.csv')
dataframe_file.head()

,Date,BP us dollar of the day,US Domestic Oil Price (US $) PET_F000000__3_A,U.S. Landed Costs of Saudi Arabian Light Crude Oil isa4990008a,BP Arabian Light posted at Ras Tanura.,Crude Oil Prices: Brent – Europe ACOILBRENTEU,Crude Oil Prices: West Texas Intermediate (WTI) ACOILWTICO,Imported Crude Oil Price (refiner average imported crude oil acquisition cost) (PET.R1300____3.A),"U.S. Crude Oil Domestic Acquisition Cost by Refiners, Annual (PET.R1200____3.A)",Dubai Crude Oil Price opendataforafrica.org/IMFPCP2014Jan
0,1861,0.49,0.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1862,1.05,1.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1863,3.15,3.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1864,8.06,8.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1865,6.59,6.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Use REST APIs to retrieve online data <a name="restapi"></a>

Generally speaking, the vast amount of data available from the Internet is accessed not through files, but through REST APIs. Fortunately, because a notebook cell can contain whatever code you like, you can use code to send requests and receive JSON data. You can then convert that JSON into whatever format you want to use, such as a pandas dataframe.

The following example is taken from https://dev.socrata.com/foundry/data.cityofnewyork.us/gkne-dk5s, which provides a dataset for 2014 New York Taxis. The sodapy library is used in the code.

In [3]:
!pip install sodapy

  Using cached https://files.pythonhosted.org/packages/ff/17/5cbb026005115301a8fb2f9b0e3e8d32313142fe8b617070e7baad20554f/requests-2.20.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/56/9d/1d02dd80bc4cd955f98980f28c5ee2200e1209292d5f9e9cc8d030d18655/certifi-2018.10.15-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Found existing installation: requests 2.14.2
    Uninstalling requests-2.14.2:
      Successfully uninstalled requests-2.14.2


The following code cell produces a warning about requests being throttled because you're not using an API key. This warning can be safely ignored.

In [4]:
# From https://dev.socrata.com/foundry/data.cityofnewyork.us/gkne-dk5s
# (select the "Python pandas") tab under "Code Snippets".

import pandas
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 20 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("gkne-dk5s", limit=20)

# Convert to pandas DataFrame
dataframe_rest1 = pandas.DataFrame.from_records(results)
print(dataframe_rest1)

           dropoff_datetime    dropoff_latitude    dropoff_longitude  \
0   2014-11-18T13:15:00.000  40.760216999999997  -73.958607000000001   
1   2014-09-26T13:52:00.000  40.734214999999999  -73.999645000000001   
2   2014-05-22T07:49:33.000  40.741612000000003  -73.987409999999997   
3   2014-10-27T21:07:18.000           40.781025  -73.979944000000003   
4   2014-04-18T14:06:36.000  40.777411000000001  -73.986192000000003   
5   2014-11-18T16:55:51.000            40.74973  -73.991425000000007   
6   2014-02-12T10:15:41.000           40.741377  -73.985422999999997   
7   2014-01-01T20:54:21.000           40.659993  -73.802232000000004   
8   2014-03-20T07:08:00.000  40.750722000000003  -73.989626999999999   
9   2014-12-14T09:33:06.000  40.768510999999997  -73.981611000000001   
10  2014-05-14T22:33:08.000  40.783008000000002           -73.948487   
11  2014-06-01T03:38:08.000  40.767149000000003           -73.996257   
12  2014-09-03T18:33:53.000  40.732773999999999  -73.97997800000

A general data request can just use the requests module:

In [5]:
import pandas
import requests

data_url = 'https://data.cityofnewyork.us/resource/gkne-dk5s.json'

# General data request; include other API keys and credentials as needed in the data argument
response = requests.get(data_url, data={"limit" : "20"})

if response.status_code == 200:
    dataframe_rest2 = pandas.DataFrame.from_records(response.json())    
    print(dataframe_rest2)

            dropoff_datetime    dropoff_latitude    dropoff_longitude  \
0    2014-11-18T13:15:00.000  40.760216999999997  -73.958607000000001   
1    2014-09-26T13:52:00.000  40.734214999999999  -73.999645000000001   
2    2014-05-22T07:49:33.000  40.741612000000003  -73.987409999999997   
3    2014-10-27T21:07:18.000           40.781025  -73.979944000000003   
4    2014-04-18T14:06:36.000  40.777411000000001  -73.986192000000003   
5    2014-11-18T16:55:51.000            40.74973  -73.991425000000007   
6    2014-02-12T10:15:41.000           40.741377  -73.985422999999997   
7    2014-01-01T20:54:21.000           40.659993  -73.802232000000004   
8    2014-03-20T07:08:00.000  40.750722000000003  -73.989626999999999   
9    2014-12-14T09:33:06.000  40.768510999999997  -73.981611000000001   
10   2014-05-14T22:33:08.000  40.783008000000002           -73.948487   
11   2014-06-01T03:38:08.000  40.767149000000003           -73.996257   
12   2014-09-03T18:33:53.000  40.732773999999999  -

## Query an Azure SQL database  <a name="azuresql"></a>

You can access SQL Server databases with the assistance of the pyodbc library.

For this article, follow the instruction in [Use Python to query an Azure SQL database](https://docs.microsoft.com/azure/sql-database/sql-database-connect-query-python) to create a database containing AdventureWorks data. The code below, taken from that article, queries the database using pyodbc.

**IMPORTANT**: To run the code, you must change the placeholders to provide identify your specific SQL Server instance and to provide your credentials.

In [6]:
!pip install pyodbc

In [7]:
import pyodbc

# Uncomment and modify these four variables for your particular instance
# Follow https://docs.microsoft.com/azure/sql-database/sql-database-connect-query-python to create a suitable database.
# server = 'your_server.database.windows.net'
# database = 'your_database'
# username = 'your_username'
# password = 'your_password'


driver= '{ODBC Driver 13 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

cursor.execute("SELECT TOP 20 pc.Name as CategoryName, p.name as ProductName FROM [SalesLT].[ProductCategory] pc JOIN [SalesLT].[Product] p ON pc.productcategoryid = p.productcategoryid")
row = cursor.fetchone()

while row:
    print (str(row[0]) + " " + str(row[1]))
    row = cursor.fetchone()

Road Frames HL Road Frame - Black, 58
Road Frames HL Road Frame - Red, 58
Helmets Sport-100 Helmet, Red
Helmets Sport-100 Helmet, Black
Socks Mountain Bike Socks, M
Socks Mountain Bike Socks, L
Helmets Sport-100 Helmet, Blue
Caps AWC Logo Cap
Jerseys Long-Sleeve Logo Jersey, S
Jerseys Long-Sleeve Logo Jersey, M
Jerseys Long-Sleeve Logo Jersey, L
Jerseys Long-Sleeve Logo Jersey, XL
Road Frames HL Road Frame - Red, 62
Road Frames HL Road Frame - Red, 44
Road Frames HL Road Frame - Red, 48
Road Frames HL Road Frame - Red, 52
Road Frames HL Road Frame - Red, 56
Road Frames LL Road Frame - Black, 58
Road Frames LL Road Frame - Black, 60
Road Frames LL Road Frame - Black, 62


## Use Azure Storage<a name="tablestorage"></a>

Azure Storage provides several different types of non-relational storage, depending on the type of data you have and how you need to access it:

- Table Storage: provides low-cost, high-volume storage for tabular data, such as collected sensor logs, diagnostic logs, and so on.
- Blob storage: provides file-like storage for any type of data.

Azure CosmosDB is also a form of non-relational storage for JSON documents; see the [Other Azure databases](#otherdbs) section for more information on CosmosDB and a variety of other options.

### Prerequisites

**IMPORTANT**: To run this code, you must create your own Azure Storage account and specify your account name and key in the variables below. For more information, see the following articles:

- [Create a storage account](https://docs.microsoft.com/azure/storage/common/storage-quickstart-create-account?tabs=portal)
- [Copy your credentials from the Azure portal](https://docs.microsoft.com/azure/storage/blobs/storage-quickstart-blobs-python#copy-your-credentials-from-the-azure-portal). Put simply, on the Azure portal, go to the storage account and navigate to **Settings** > **Access keys**. Then copy either **key1** or **key2** and paste into the applicable code cells in this section.
- Install the azure-storage library (used for tables and blobs), which is done with the following code cell.


In [8]:
!pip install azure-storage

    100% |████████████████████████████████| 194kB 7.9MB/s ta 0:00:01


### Table storage

The following code creates a table in a specified Azure Storage account, then adds rows, removes rows, and queries data.

In [11]:
# Modify these variables with your specific values obtained in the Prerequisites section
azure_storage_account_name = "your_storage_account"
azure_storage_account_key = "your_access_key"

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("Provide your specific name and key for your Azure Storage account--see the Prerequisites section earlier.")

In [12]:
from azure.storage.table import TableService
import IPython

# Connect to the table - change the placeholders to your specific names
table_service = TableService(azure_storage_account_name, azure_storage_account_key)

# Create a table
table_name = 'azurenotebookstesttable'
table_service.create_table(table_name)

# Insert entities into the table
entity = {'PartitionKey': 'testItems', 'RowKey': '0', 'age':1}
table_service.insert_entity(table_name, entity)
table_service.insert_entity(table_name, {'PartitionKey': 'testItems', 'RowKey': '10', 'age':2, 'eyecolor':'blue'})

# Query the table
queried_entities = table_service.query_entities(table_name, filter="PartitionKey eq 'testItems'")
print('=== Queried rows after inserts ===')
IPython.display.display_pretty([i for i in queried_entities])

# Delete an entity by using its partition and row key.
table_service.delete_entity(table_name, 'testItems', '0')
                                         
# Query again to show that the entity was removed
queried_entities = table_service.query_entities(table_name, filter="PartitionKey eq 'testItems'")
print('=== Queried rows after delete ===')
IPython.display.display_pretty([i for i in queried_entities])

# Clean up resources
table_service.delete_table('azurenotebookstesttable')

=== Queried rows after inserts ===


[{'PartitionKey': 'testItems',
  'RowKey': '0',
  'Timestamp': datetime.datetime(2018, 11, 27, 16, 4, 55, 866146, tzinfo=tzlocal()),
  'age': 1,
  'etag': 'W/"datetime\'2018-11-27T16%3A04%3A55.866146Z\'"'},
 {'PartitionKey': 'testItems',
  'RowKey': '10',
  'Timestamp': datetime.datetime(2018, 11, 27, 16, 4, 56, 53278, tzinfo=tzlocal()),
  'age': 2,
  'etag': 'W/"datetime\'2018-11-27T16%3A04%3A56.0532785Z\'"',
  'eyecolor': 'blue'}]

=== Queried rows after delete ===


[{'PartitionKey': 'testItems',
  'RowKey': '10',
  'Timestamp': datetime.datetime(2018, 11, 27, 16, 4, 56, 53278, tzinfo=tzlocal()),
  'age': 2,
  'etag': 'W/"datetime\'2018-11-27T16%3A04%3A56.0532785Z\'"',
  'eyecolor': 'blue'}]

True

## Access Azure Blobs <a name="blobs"></a>

Blobs store file-like data, which can be private or public. 

The code below demonstrates private keys first. It creates a container, then creates a blob, then reads that blob.

The [shared access](#sharedaccess) section then demonstrates a shared access signature for public read-only access.

You can also put content into blobs using [AzCopy](https://azure.microsoft.com/en-us/documentation/articles/storage-use-azcopy/).

In [14]:
# Modify these variables with your specific values obtained in the Prerequisites section
azure_storage_account_name = "your_storage_account"
azure_storage_account_key = "your_access_key"

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("Provide your specific name and key for your Azure Storage account--see the Prerequisites section earlier.")

In [15]:
from azure.storage.blob import BlockBlobService

# Connect to our blob via the BlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)

# Create a container
blob_service.create_container('azure-notebooks-data')

# Insider a container, create other containers or blobs
blob_service.create_blob_from_text('azure-notebooks-data', 'sample.txt', 'Hello, Blobs! This is content for the sample.txt file.')

# You can list containers and blobs
containers = blob_service.list_containers()
blobs = blob_service.list_blobs('azure-notebooks-data')

# Read a blob from and get the text; the copy is stored in the Azure Notebooks project
blob_service.get_blob_to_path('azure-notebooks-data', 'sample.txt', 'sample.txt')

# Clean up the created blob and container
blob_service.delete_blob('azure-notebooks-data', 'sample.txt')
blob_service.delete_container('azure-notebooks-data')

True

Display the contents of the sample blob file:

In [16]:
!cat sample.txt

Hello, Blobs! This is content for the sample.txt file.

## Provide public read-only access to Azure Storage through shared access signatures <a name="sharedaccess"></a>

Sometimes you want to share data from Azure Storage without providing editing capabilities. Shared Access Signatures allow you to share your data and provide whatever level of control you want to the receiver.

The code below creates a shared access signature with read permissions for a table (it also works with blobs). The code then demonstrates the ability to read but not write. Additional permissions are also necessary to query.

In [17]:
# Modify these variables with your specific values obtained in the Prerequisites section
azure_storage_account_name = "your_storage_account"
azure_storage_account_key = "your_access_key"

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("Provide your specific name and key for your Azure Storage account--see the Prerequisites section earlier.")

In [19]:
# Create a container and a blob in that container
from azure.storage.blob import BlockBlobService

# Connect to our blob via the BlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)

# Create a container
blob_service.create_container('azure-notebooks-data')

# Insider a container, create other containers or blobs
blob_service.create_blob_from_text('azure-notebooks-data', 'sample.txt', 'Hello, Blobs! This is content for the sample.txt file.')


In [20]:
# Create a shared access signature
from azure.storage.blob.models import BlobPermissions
from datetime import datetime, timedelta

sas_token = blob_service.generate_blob_shared_access_signature(
    'azure-notebooks-data',
    'sample.txt',
    BlobPermissions.READ,
    datetime.utcnow() + timedelta(hours=1)
)

print(sas_token)

sr=b&se=2018-11-27T17%3A05%3A48Z&sv=2017-04-17&sp=r&sig=yK0jkCmRIanitFE%2BtYtzvAXES0j/oJrfU1S7q2hTNho%3D


In [21]:
# Create a service and use the shared access signature
sas_blob_service = BlockBlobService(account_name=azure_storage_account_name, sas_token=sas_token, )

sas_blob_service.get_blob_to_text('azure-notebooks-data', 'sample.txt').content

'Hello, Blobs! This is content for the sample.txt file.'

In [22]:
# Clean up the created blob and container
blob_service.delete_blob('azure-notebooks-data', 'sample.txt')
blob_service.delete_container('azure-notebooks-data')

True

## Query other Azure databases  <a name="otherdbs"></a>

Azure provides a number of other database types that you can use. The articles below provide guidance for accessing those databases from Python:

- Azure Cosmos DB (fully-indexed NoSQL store for JSON documents):
  - [Build a SQL API app with Python](https://docs.microsoft.com/azure/cosmos-db/create-sql-api-python)
  - [Build a Flask app with the MongoDB API](https://docs.microsoft.com/azure/cosmos-db/create-mongodb-flask)
  - [Create a graph database using Python and the Gremlin API](https://docs.microsoft.com/azure/cosmos-db/create-graph-python)
  - [Build a Cassandra app with Python and Azure Cosmos DB](https://docs.microsoft.com/azure/cosmos-db/create-cassandra-python)
  - [Build a Table API app with Python and Azure Cosmos DB](https://docs.microsoft.com/azure/cosmos-db/create-table-python)
- [Azure Database for PostgreSQL: Use Python to connect and query data](https://docs.microsoft.com/azure/postgresql/connect-python)
- [Quickstart: Use Azure Redis Cache with Python](https://docs.microsoft.com/azure/redis-cache/cache-python-get-started)
- [Azure Database for MySQL: Use Python to connect and query data](https://docs.microsoft.com/azure/mysql/connect-python)
- [Azure Data Factory](https://azure.microsoft.com/en-us/services/data-factory/)
  - [Copy Wizard for Azure Data Factory](https://azure.microsoft.com/en-us/updates/code-free-copy-wizard-for-azure-data-factory/)

Note that for CosmosDB, you can use the [azure-cosmosdb-table](https://pypi.org/project/azure-cosmosdb-table/) library (`!pip install azure-cosmosdb-table`).